In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
cols = ['engine_id', 'cycle', 'op1', 'op2', 'op3'] + \
       [f'sensor_{i}' for i in range(1, 22)]

df = pd.read_csv(
    '../data/raw/train_FD001.txt',
    sep=' ',
    header=None,
    names=cols
)


In [4]:
max_cycles = df.groupby('engine_id')['cycle'].max()

df['RUL'] = df.apply(
    lambda row: max_cycles[row['engine_id']] - row['cycle'],
    axis=1
)


In [5]:
drop_cols = ['op3', 'sensor_1', 'sensor_5', 'sensor_10']  # example
df_clean = df.drop(columns=drop_cols)


In [6]:
RUL_CAP = 130
df_clean['RUL'] = df_clean['RUL'].clip(upper=RUL_CAP)


In [9]:
window = 5

for col in [c for c in df_clean.columns if 'sensor' in c]:
    df_clean[f'{col}_rollmean'] = (
        df_clean.groupby('engine_id')[col]
        .rolling(window)
        .mean()
        .reset_index(level=0, drop=True)
    )


In [10]:
df_clean = df_clean.dropna()
